In [2]:
import gym
from gym import spaces

In [ ]:
'''
Each environment must implement the following gym interface

class CustomEnv(gym.Env):
    metadata = {'render.modes':['human']}
    
    def __init__(self, arg1, arg2, ...):
        super(CustomEnv, self).__init__()
        
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        
        self.observation_space = spaces.Box(low=0,high=255,shape=(HEIGHT, WIDTH, N_CHANNELS), dype=np.uint8)
        
    def step(self,action):
        pass
        
    def reset(self):
        pass
        
    def render(self,mode='human',close=False):
        pass
            
'''

In [ ]:
class StockTradingEnvironment(gym.Env):
    
    metadata = {'render.modes':['human']}
    
    def __init__(self, df):
        super(StockTradingEnv, self).__init__()
        self.df = df
        sef.reward_range = (0, MAX_ACCOUNT_BALANCE)
        
        self.action_space = spaces.Box(low=np.array([0,0]), high=np.array([3,1]), dtype=np.float16)
        
        self.observation_space = spaces.Box(low=0,high=1,shape(6,6),dtype=np.float16)
        
    def reset(self):
        pass
    
    def step(self,action):
        pass
    
    def render(self,mode='humnan',close=False)
    pass